<a href="https://colab.research.google.com/github/AndrewDavidRatnam/YoutubeVideoAnalysis/blob/main/Instagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import time
from google.colab import userdata


#CONFIG

In [ ]:
IG_USER_ID=userdata.get('IG_USER_ID') #check if this global or changes with each session
access_token = userdata.get('IG_access_token')#short token
app_secret = userdata.get('IG_SECRET') #WTF is this lol
app_id = userdata.get('IG_APP')
version = ""#get latest version from docs MFs update it like crazy

## LONG ACCESS TOKEN
Roundabout way to get the token but you don't trust google with meta data and vice versa lol


In [ ]:
url=f"""https://graph.facebook.com/{version}/oauth/access_token?
grant_type=fb_exchange_token&client_id=(app_id)&client_secret={app_secret}
&fb_exchange_token={access_token}"""
response = requests.get(url)
long_access_token = response.json()["acess_token"]

#Info

like urls and stuff
- required_params: "(followers_count,media_count)" etc
- 'https://graph.facebook.com/{version}/{IG_USER_ID}/media_publish'
- 'https://graph.facebook.com/{version}/{IG_USER_ID}/media'
- 'https://graph.facebook.com/{version}/{IG_USER_ID}/fields=business_discovery.username({username}){required_params}&access_token={long_access_token}'

# CODE

## Account Details

### IDEK

In [5]:
def get_data(username,required_params):
  #required_params: "(followers_count,media_count)"
  url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/fields=business_discovery.username({username}){required_params}&access_token={long_access_token}'
  response = requests.get(url)
  metadata = response.json()
  followers_count = metadata["business_discovery"]["followers_count"]
  media_count = metadata["business_discovery"]["media_count"]
  data = json.dumps(metadata["business_discovery"],indent=2)
  # print(data)
  return data




### ACCOUNT DETAILS
- [DOCS IG USER](https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user)
- [DOCS IG MEDIA](https://developers.facebook.com/docs/instagram-platform/reference/instagram-media)
<br>
##Fields
Public fields can be returned by an edge using field expansion. Only a few fields will be available for accessing Page-backed Instagram accounts.
***

Field Name	Description
alt_text
Public

Descriptive text for images, for accessibility.
***

biography
Public

Profile bio text.
***

followers_count
Public

Total number of Instagram users following the user.
***

follows_count

Total number of Instagram users the user follows.
***

has_profile_pic

Indicates whether your app user's Instagram professional account has a profile picture.
***

id
Public

App-scoped User ID. Available for Page-backed Instagram accounts.
***

is_published

Indicates whether your app user's Instagram account is published. Available for Page-backed Instagram accounts.
***

legacy_instagram_user_id

Your app user's Instagram ID that was created for Marketing API endpoints for v21.0 and older. Available for Page-backed Instagram accounts.
***

media_count
Public

Total number of IG Media published on your app user's account.
***

name

Your app user's Instagram profile name.
***

profile_picture_url

Your app user's Instagram profile picture URL.
***

shopping_product_tag_eligibility

Returns true if your app user has set up an Instagram Shop and is therefore eligible for product tagging, otherwise returns false.
***

username
Public

Your app user's Instagram profile username.
***

website
Public

Your app user's website URL.
***

##Edges
Edge	Description
***

agencies

A list of businesses that can advertise for this Instagram professional account.
***

authorized_adaccounts

Ad accounts that can advertise for this Instagram professional account.
***

business_discovery

Get data about other Instagram Business or Instagram Creator IG Users.
***

connected_threads_user

Represents a Threads account connected to an Instagram account.
***

content_publishing_limit

Represents an IG User's current content publishing usage.
***

insights

Represents social interaction metrics on an IG User.
***

instagram_backed_threads_user

Represents a Threads account backed by an Instagram account.
***

live_media

Represents a collection of live video IG Media on an IG User.
***

media

Represents a collection of IG Media on an IG User.
***

media_publish

Publish an IG Container on an Instagram Business IG User.
***

mentions

Create an IG Comment on an IG Comment or captioned IG Media that an IG User has been @mentioned in by another Instagram user.
***

mentioned_comment

Get data on an IG Comment in which an IG User has been @mentioned by another Instagram user.
***

mentioned_media

Get data on an IG Media in which an IG User has been @mentioned in a caption by another Instagram user.
***

recently_searched_hashtags

Get IG Hashtags that an IG User has searched for within the last 7 days.
***

stories

Represents a collection of story IG Media objects on an IG User.
***

tags

Represents a collection of IG Media in which an IG User has been tagged by another Instagram user.
***

upcoming_events

A list of events this Instagram professional account is hosting.
***


CAN USE TO GET INFO ON ANY BUSSINESS ACCOUNT
- media returns 25 top, max around 500 ```media.limit(limit)(thumbnail_url...... etc)```

In [ ]:
required_params: "(name,website,biography,followers_count,follows_count,media_count,profile_picture_url,media(thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink))"
def get_info(username,required_params):
  url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/fields=business_discovery.username({username}){required_params}&access_token={long_access_token}'
  response = requests.get(url)
  metadata = response.json()
  return(json.dumps(metadata, indent=2))

SAY WITH THE MEDIA URL WE WANT TO DOWNLOAD THE MEDIA TO ANALYZE(COPY THE IDEA "BAR FOR BAR")

In [3]:
# metadata["business_discovery"]["media"]["data"][0]["media_url"]
# metadata["business_discovery"]["media"]["data"][0]["thumbnail_url"] #0 used to get first media data
def download_media(post_link,thumbnail_link,destination_folder_video,destination_folder_thumbnail):
  response = requests.get(post_link)
  response_ = requests.get(thumbnail_link)

  with open(destination_folder_video,"wb") as f:
    f.write(response.content)
  with open(destination_folder_thumbnail,"wb") as f:
    f.write(response.content)

PAGINATED RESULTS

{media.limit(1).after(your_after_code){thumbnail_url..etc}}

In [4]:
metadata = {}# response you get from GET
after_code = metadata["business"]["aftercode"]
required_params = "media.after("+ after_code +"){thumbnail_url...etc}}"
username = ""
data = get_info(username,required_params)

LOOP OVER AND GET ALL POSTS IF YOUR BANDWIDTH DOG WATER
- also carousel will return as one object after the other, so one pic is one media link i think

### POST AN IMAGE

- image must be jpg format only

<h3>Making your media public<h3>

- Start a HTTP Server
- Run ngrok on your directory
- get public url

***
<h3>Publish and Post an Image<h3>

- Create container using a public url of the media
- Publish the container with media
- Get the post url for futher data Analysis
***

In [2]:
import http.server
import datetime
import socketserver
import threading
import subprocess
import time
import json

import requests

PORT = 8000
Handler = httpserver.SimpleHTTPRequestHandler
httpd = sockerserver.TCPServer(("", PORT), Handler)
thread = threading.Thread(target = httpd.server_forever)
thread.start()

ngrok = subprocess.Popen([path_to_ngrok_exe,"http",str(PORT)], stdout = subprocess.PIPE)

time.sleep(2) #so ngrok can start up

response = requests.get("http://localhost:4040/api/tunnels") #im confused here lol but if it works it works
public_url = response.json()["tunnels"][0]["public_url"]

path_to_media = ""
post_url = public_url + path_to_media

caption = "67!?"


#Create a Container
create_post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media'
create_post_payload = {
      'image_url':post_url,
      'caption': caption,
      'access_token': access_token,
      "user_tags":"[{'username':'user_name','x':0.5,'y':0.5}]",
      "collaborators":['username'],

  }

# create_post_payload = {
#       'image_url':post_url,
#       'media_type':"STORIES" #"REELS"
#       'caption': caption,
#       'access_token': access_token,
#       "user_tags":"[{'username':'user_name','x':0.5,'y':0.5}]", #FOR REELS ONLY
#       "collaborators":['username'], #FOR REELS ONLY

#   }

response = requests.post(create_post_url, data=create_post_payload)
time.sleep(2) #it takes time to create
data = response.json()
"""
  below code can be reused, only container_id is required(variable)
  to publish it's the same(carousel also same just change media_type)
"""
if 'id' in data:
  creation_id = data['id']
  post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media_publish'
  post_payload = {
      'creation_id':creation_id,
      'access_token' : access_token
    }
  response = requests.post(post_url, data=post_payload)
  time.sleep(2) #it takes time to create
  data = response.json()
  media_id = data["id"]

else:
  print("Something went wrong")



url = f'https://graph.facebook.com/{version}/{media_id}'
payload ={
    "fields":"comments_count, like_count, media_url, permalink",
    "access_token" = access_token
}
response = requests.get(url,params=payload)
data = response.json() #media url is cdn #permalink is instagram/p/somehashedkey or instagram/reels/asdagAGD

ngrok.terminate()
httpd.shutdown() #closing in thread in hectick, just get PID and kill -9 that shii

CHECK RATE LIMIT (50 posts a day lmao)


In [1]:
url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/publishing_limit'
payload = [
    "fields":"quota_usage, config",
    "access_token":access_token
]
response = requests.get(url, params=payload)
metadata = response.json()
print(json.dumps(metadata))

## POST REEL

- mov or mp4 format only and not larger and a gb
- thumbnail in jpeg and not more than 8mb
- max duration 15 mins, 3 secs minimum
- content must be hosted on a publicly accessiblr server
- Only 50 API-published in 24 hours(rolling)
- caption limitations(2200 characters, 30 hastags, 20@tags)

- For Story: mov or mp4 not more tan 100mb, max 60seconds, publicly accessiblr

## POST IMAGE

In [ ]:

#JUST USE NGROK LOL
def post_image_to_instagram(image, access_token,version,caption, gdrive_image_url): #see if you post directly from gdrive(better)(nah the redirect and token stuff)
  create_post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media'
  create_post_payload = {
      'image_url':gdrive_image_url,
      'caption': caption,
      'access_token': access_token
  }
  request = requests.post(create_post_url, data=create_post_payload)
  print(requests.text)
  print("Media uploaded Sucessfully") #created a container

  results = json.loads(request.text)
  """
  below code can be reused, only container_id is required(variable)
  to publish it's the same(carousel also same just change media_type)
  """
  if 'id' in results:
    container_id = results['id']
    post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media_publish'
    post_payload = {
        'creation_id':container_id,
        'access_token' : access_token
    }
    r = requests.post(post_url, data=post_payload)
    print(r.text)
    print("image published to instagram")

  else:
    print("Something went wrong")


## POST REELS

In [ ]:
def upload_reel_to_instagram(, access_token,version,caption, gdrive_video_url): #see if you post directly from gdrive(better)
  create_reel_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media'
  create_reel_payload = {
      'media_type':'REELS',
      'image_url': gdrive_video_url,
      'caption': caption,
      'access_token': access_token
  }
  request = requests.post(create_reel_url, data=create_reel_payload)
  print(requests.text)
  print("Reel uploaded Sucessfully") #created a container

  results = json.loads(request.text)

  # if 'id' in results:
  #   container_id = results['id']
  #   post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media_publish'
  #   post_payload = {
  #       'creation_id':container_id,
  #       'access_token' : access_token
  #   }
  #   r = requests.post(post_url, data=post_payload)
  #   print(r.text)
  #   print("image published to instagram")

  # else:
  #   print("Something went wrong")

  return results

def status_code(ig_container_id, access_token):
  graph_url = f'https://graph.facebook.com/{version}/{ig_container_id}'
  param = {}
  param['access_token']=access_token
  param['fields']='status_code' #what? double check this
  response=requests.get(graph_url,params=param) #checking the reel upload container status code
  response = response.json()
  return(response['status_code'])

def publish_reel_instagram(results, access_token):
  if 'id' in results:
    container_id = results['id']
    post_url = f'https://graph.facebook.com/{version}/{IG_USER_ID}/media_publish'
    post_payload = {
        'creation_id':container_id,
        'access_token' : access_token
    }
    r = requests.post(post_url, data=post_payload)
    print(r.text)
    print("video published to instagram")

  else:
    print("Something went wrong")
